In [ ]:
# ===== GÉNÉRATEUR COMPLET PITIÉ-SALPÊTRIÈRE (1 800 LITS) =====
import pandas as pd
import numpy as np
from datetime import datetime
import os

# 1. LITS PAR SERVICE (ATIH calibré 1 800 lits) 
lits_par_service = {
    'Réanimation': 81,
    'Urgences': 108, 
    'Médecine': 396,
    'Chirurgie': 306,
    'Cardiologie': 198,
    'Neurologie': 180,
    'Maladies_infectieuses': 126,
    'Pédiatrie': 144,
    'Autres': 261
}

# 2. PERSONNEL PAR SERVICE (DREES ratios × lits réels)
personnel_par_service = {
    'Médecine': {'total': 724, 'medecins': 99, 'infirmiers': 269, 'aides': 178},
    'Chirurgie': {'total': 561, 'medecins': 77, 'infirmiers': 208, 'aides': 138},
    'Réanimation': {'total': 112, 'medecins': 20, 'infirmiers': 50, 'aides': 15},
    'Cardiologie': {'total': 404, 'medecins': 50, 'infirmiers': 135, 'aides': 89},
    'Neurologie': {'total': 376, 'medecins': 45, 'infirmiers': 122, 'aides': 81},
    'Maladies_infectieuses': {'total': 270, 'medecins': 32, 'infirmiers': 86, 'aides': 57},
    'Urgences': {'total': 225, 'medecins': 27, 'infirmiers': 72, 'aides': 48},
    'Pédiatrie': {'total': 324, 'medecins': 40, 'infirmiers': 108, 'aides': 72},
    'Autres': {'total': 598, 'medecins': 60, 'infirmiers': 174, 'aides': 132}
}

# 3. ÉQUIPEMENTS PAR SERVICE (Pitié 2015 + ratios) 
equipements_par_service = {
    'Réanimation': {
        'Respirateurs': 120, 'Moniteurs': 90, 'Perfuseurs': 100,
        'Défibrillateurs': 25, 'Ventilateurs_NIV': 80, 'Échos': 10
    },
    'Médecine': {
        'Moniteurs': 180, 'Perfuseurs': 450, 'Échos': 30, 
        'Défibrillateurs': 20, 'Respirateurs': 20
    },
    'Chirurgie': {
        'Moniteurs': 60, 'Perfuseurs': 250, 'Échos': 25,
        'Défibrillateurs': 20, 'Respirateurs': 10
    },
    'Imagerie': {
        'IRM': 5, 'Scanners': 4, 'Échos': 15
    },
    'Urgences': {
        'Moniteurs': 25, 'Perfuseurs': 35, 'Défibrillateurs': 15
    }
}

# 4. STOCKS MÉDICAMENTS (ANSM + Décret 2021-349) 
stocks_medicaments = {
    'Paracétamol_1g': {'conso_base': 2.5, 'stock_init': 162000, 'securite_jours': 30},
    'Antibiotiques': {'conso_base': 1.8, 'stock_init': 29160, 'securite_jours': 45},
    'Morphine_IV': {'conso_base': 0.4, 'stock_init': 6480, 'securite_jours': 45},
    'Heparine': {'conso_base': 6, 'stock_init': 97200, 'securite_jours': 30},
    'Insuline': {'conso_base': 60, 'stock_init': 194400, 'securite_jours': 30}
}

# PARAMÈTRES GLOBAUX
date_range = pd.date_range('2024-01-01', '2025-12-31', freq='D')
os.makedirs('data', exist_ok=True)

taux_occ_moyen = {
    'Réanimation': 0.87, 'Urgences': 0.91, 'Médecine': 0.81, 'Chirurgie': 0.83,
    'Cardiologie': 0.79, 'Neurologie': 0.78, 'Maladies_infectieuses': 0.70,
    'Pédiatrie': 0.74, 'Autres': 0.77
}

taux_util_equip = {
    'Respirateurs': 0.65, 'Moniteurs': 0.80, 'Perfuseurs': 0.75,
    'IRM': 0.92, 'Scanners': 0.88, 'Échos': 0.70, 'Défibrillateurs': 0.60
}

multiplicateurs_service = {
    'Réanimation': 3.0, 'Maladies_infectieuses': 2.2, 'Urgences': 1.8,
    'Chirurgie': 1.5, 'Médecine': 1.0, 'Cardiologie': 1.2, 'Pédiatrie': 0.8
}

print("GÉNÉRATEUR PITIÉ-SALPÊTRIÈRE 1.8K LITS / 3.2K ETP")
print("Génération 2023-2025 (1095 jours)...")

# ===== 1. LITS ET OCCUPATION =====
lits_data = []
for date in date_range:
    for service, lits_total in lits_par_service.items():
        taux = taux_occ_moyen[service]
        
        # Saisonnalité DGOS
        if date.month in [12, 1, 2]: taux *= 1.05  # Hiver
        elif date.month in [7, 8]: taux *= 0.97     # Été
        if date.weekday() >= 5: taux *= 0.98        # Weekend
        
        taux *= np.random.normal(1, 0.03)
        taux = np.clip(taux, 0.5, 0.99)
        
        lits_occ = int(lits_total * taux)
        
        lits_data.append({
            'date': date, 'service': service, 'lits_totaux': lits_total,
            'lits_occupes': lits_occ, 'taux_occupation': round(taux, 3)
        })

df_lits = pd.DataFrame(lits_data)
df_lits.to_csv('data/lits_et_occupation.csv', index=False)
print(f"lits_et_occupation.csv ({len(df_lits):,} lignes)")

# ===== 2. PERSONNEL =====
perso_data = []
for date in date_range:
    for service, effectifs in personnel_par_service.items():
        for categorie, eff_total in effectifs.items():
            taux_abs = 0.085
            if date.month in [7, 8]: taux_abs = 0.15
            elif date.month == 12 and date.day >= 20: taux_abs = 0.12
            
            taux_abs += np.random.normal(0, 0.01)
            taux_abs = np.clip(taux_abs, 0.03, 0.25)
            
            eff_present = int(eff_total * (1 - taux_abs))
            
            perso_data.append({
                'date': date, 'service': service, 'categorie': categorie,
                'effectif_total': eff_total, 'effectif_present': eff_present,
                'taux_absence': round(taux_abs, 3)
            })

df_perso = pd.DataFrame(perso_data)
df_perso.to_csv('data/personnel_disponible.csv', index=False)
print(f"  personnel_disponible.csv ({len(df_perso):,} lignes)")

# ===== 3. ÉQUIPEMENTS =====
equip_data = []
for date in date_range:
    for service, equip_list in equipements_par_service.items():
        for equip, total in equip_list.items():
            taux = taux_util_equip.get(equip, 0.75)
            
            # Corrélations saisonnières
            if service == 'Réanimation' and date.month in [12,1,2]:
                taux *= 1.15
            if equip == 'IRM': taux = 0.92  # Planning serré
            
            taux *= np.random.normal(1, 0.05)
            taux = np.clip(taux, 0.4, 0.98)
            
            en_service = int(total * taux)
            
            equip_data.append({
                'date': date, 'service': service, 'equipement': equip,
                'quantite_totale': total, 'en_service': en_service,
                'disponible': total - en_service, 'taux_utilisation': round(taux, 3)
            })

df_equip = pd.DataFrame(equip_data)
df_equip.to_csv('data/equipements_medicaux.csv', index=False)
print(f"  equipements_medicaux.csv ({len(df_equip):,} lignes)")

# ===== 4. STOCKS MÉDICAMENTS =====
stocks_actuels = {k: v['stock_init'] for k, v in stocks_medicaments.items()}
stocks_data = []

# Pré-calcul lits occupés par jour/service pour éviter l'erreur
lits_occupes_par_jour = {}
for date in date_range:
    jour_data = df_lits[df_lits['date'] == date].set_index('service')['lits_occupes'].to_dict()
    lits_occupes_par_jour[date] = jour_data

for date in date_range:
    # Consommation = Σ(lits occupés × multiplicateur service)
    conso_totale = 0
    for service, lits_occ in lits_occupes_par_jour[date].items():
        mult = multiplicateurs_service.get(service, 1.0)
        conso_totale += lits_occ * mult
    
    for medoc, params in stocks_medicaments.items():
        conso_jour = conso_totale * params['conso_base'] * np.random.normal(1, 0.05)
        
        # Livraisons : mardi/jeudi OU urgente si critique
        livraison = 0
        if date.weekday() in [1, 3]:  # Mardi/Jeudi
            livraison = params['stock_init'] * 0.1
        
        stock_fin = stocks_actuels[medoc] - conso_jour + livraison
        stocks_actuels[medoc] = max(0, stock_fin)
        
        alerte = stock_fin < (conso_jour * 7)  # < 7 jours consommation
        
        stocks_data.append({
            'date': date, 'medicament': medoc, 
            'conso_jour': round(conso_jour),
            'livraison': round(livraison), 
            'stock_fin': round(stock_fin),
            'alerte_rupture': alerte, 
            'jours_stock': round(stock_fin/conso_jour, 1) if conso_jour > 0 else 0
        })

df_stocks = pd.DataFrame(stocks_data)
df_stocks.to_csv('data/stocks_medicaments.csv', index=False)
print(f"   stocks_medicaments.csv ({len(df_stocks):,} lignes)")


# ===== 5. RÉSUMÉ STATISTIQUES =====
print("\n RÉSUMÉ STATISTIQUES PITIÉ-SALPÊTRIÈRE")
print(f"   Lits totaux: {sum(lits_par_service.values()):,} ({sum(lits_par_service.values())})")
print(f"   Effectifs ETP: {sum([sum(v.values())-sum(['total' in k for k in v]) for v in personnel_par_service.values()])}")
print(f"   Équipements: {sum([sum(v.values()) for v in equipements_par_service.values()])}")
print(f"   Alertes ruptures: {df_stocks['alerte_rupture'].sum()}")


🚀 GÉNÉRATEUR PITIÉ-SALPÊTRIÈRE 1.8K LITS / 3.2K ETP
📊 Génération 2023-2025 (1095 jours)...
  ✓ lits_et_occupation.csv (9,864 lignes)
  ✓ personnel_disponible.csv (39,456 lignes)
  ✓ equipements_medicaux.csv (24,112 lignes)
  ✓ stocks_medicaments.csv (5,480 lignes)

📈 RÉSUMÉ STATISTIQUES PITIÉ-SALPÊTRIÈRE
   Lits totaux: 1,800 (1800)
   Effectifs ETP: 6069
   Équipements: 1589
   Alertes ruptures: 4381

🎉 GÉNÉRATION TERMINÉE - PUSH GitHub:
   git add .
   git commit -m 'Tables synthétiques Pitié-Salpêtrière v1.0'
   git push
